In [34]:
import torch
import torch.nn as nn

In [65]:
class MobileNetV1(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(MobileNetV1, self).__init__()
        self.models = nn.Sequential(
            self.conv_bn(in_channels, 32, 2), 
            self.conv_dw(32, 64, 1),
            self.conv_dw(64, 128, 2),
            self.conv_dw(128, 128, 1),
            self.conv_dw(128, 256, 2),
            self.conv_dw(256, 256, 1),
            self.conv_dw(256, 512, 2),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 1024, 2),
            self.conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7)
        )

        self.fc_layer = nn.Linear(1024, num_classes)

    def conv_bn(self, in_channels, out_channels, stride):
        return nn.Sequential(StandardConv(in_channels, out_channels, stride))

    def conv_dw(self, in_channels, out_channels, stride):
        layers = []
        layers.append(DepthwiseConv(in_channels, in_channels, stride))
        layers.append(PointwiseConv(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.models(x)
        x = x.view(-1)
        x = self.fc_layer(x)
        return x

In [36]:
class DepthwiseConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(DepthwiseConv, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)

    def forward(self, x):
        return nn.ReLU(nn.BatchNorm2d(self.conv(x)))

In [37]:
class PointwiseConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(PointwiseConv, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, padding=0, bias=False)

    def forward(self, x):
        return nn.ReLU(nn.BatchNorm2d(self.conv(x)))

In [38]:
class StandardConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(StandardConv, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

    def forward(self, x):
        return nn.ReLU(nn.BatchNorm2d(self.conv(x)))

## Reference
- [pytorch-mobilenet-v1](https://github.com/wjc852456/pytorch-mobilenet-v1)